# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In [3]:
#load .env file
import os
from dotenv import load_dotenv, find_dotenv

# This finds the .env file in the current directory (or parent dirs)
_ = load_dotenv(find_dotenv())

# Now you can access your key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("Key loaded:", bool(OPENAI_API_KEY))  # should print True


Key loaded: True


In [4]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [5]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010. He was driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [6]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [7]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [8]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton, driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [9]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [10]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.  
Team: Renault.


Few Shots for classification.


In [12]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
#Few‑Shot Classification Experiments

!pip -q install transformers==4.43.3 torch sentencepiece accelerate
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import re, json, random, itertools
from collections import Counter, defaultdict

# --- Model (small, fast) ---
tok = AutoTokenizer.from_pretrained("google/flan-t5-small")
mdl = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
gen = pipeline("text2text-generation", model=mdl, tokenizer=tok, max_new_tokens=64)

def run(prompt: str) -> str:
    return gen(prompt)[0]["generated_text"].strip()

# --- Tiny labeled dataset (balanced-ish) ---
data = [
    ("Absolutely love it — sturdy and worth the price.", "Positive"),
    ("Terrible experience; broke on day two.", "Negative"),
    ("It’s okay. Does the job, nothing special.", "Neutral"),
    ("Exceeded expectations; would buy again.", "Positive"),
    ("I’m not returning it, but I wouldn’t buy it again.", "Negative"),
    ("Fine for the price, but has minor issues.", "Neutral"),
    ("Fantastic battery life and screen.", "Positive"),
    ("Support was unhelpful and slow.", "Negative"),
    ("Works as described; acceptable overall.", "Neutral"),
    ("Great quality and fast shipping.", "Positive"),
    ("Way too expensive for what it does.", "Negative"),
    ("Usable, though my partner likes it more than I do.", "Neutral"),
]

LABELS = {"Positive","Negative","Neutral"}

def normalize_label(s: str) -> str:
    """Map free-form output to one of the canonical labels; default Neutral on mismatch."""
    s = s.strip()
    # Try JSON first
    try:
        obj = json.loads(s)
        if isinstance(obj, dict) and "sentiment" in obj:
            s = str(obj["sentiment"])
    except Exception:
        pass
    s = re.split(r"[\s,.;:!?\-]+", s)[0].capitalize()
    return s if s in LABELS else "Neutral"

# ------------------------------
# Prompt Version A — Zero-shot (instruction-only, free-form)
# ------------------------------
def prompt_zero_shot(text: str) -> str:
    return f"""Classify the review sentiment as Positive, Negative, or Neutral.
Return exactly one word.

Review: "{text}"
Label:"""

# ------------------------------
# Prompt Version B — One-shot (single exemplar, one-word target)
# ------------------------------
ONE_SHOT = (
    'Review: "I love this product so much; highly recommended!"\n'
    "Label: Positive\n"
)
def prompt_one_shot(text: str) -> str:
    return f"""Classify the review sentiment as Positive, Negative, or Neutral.
Return exactly one word.

{ONE_SHOT}
Review: "{text}"
Label:"""

# ------------------------------
# Prompt Version C — Few-shot JSON (3 exemplars, strict schema)
# ------------------------------
FEW_SHOT_JSON = [
    ('Review: "It didn\'t work; very disappointed."', '{"sentiment":"Negative"}'),
    ('Review: "It works fine; nothing special."', '{"sentiment":"Neutral"}'),
    ('Review: "Amazing build quality and value."', '{"sentiment":"Positive"}'),
]
def prompt_few_shot_json(text: str) -> str:
    shots = "\n".join(f"{inp}\n{out}" for inp,out in FEW_SHOT_JSON)
    return f"""You are an expert in review sentiment.
Output valid JSON on one line: {{"sentiment":"Positive|Negative|Neutral"}}. No extra text.

{shots}
Review: "{text}"
"""

# ------------------------------
# Prompt Version D — BAD PROMPT (inconsistent labels & punctuation)
# Demonstrates a variant that tends to fail/drift.
# ------------------------------
def prompt_bad(text: str) -> str:
    return f"""Classify as positive or negative. (Note: sometimes it's neutral.)
Return the best label you think, maybe with an explanation.

Example:
Text: "It was fine."
Sentiment: Neutral.

Now classify:
Text: "{text}"
Sentiment:"""

# --- Evaluation Harness ---
def evaluate(version_name, prompt_fn, samples):
    preds, golds, rows = [], [], []
    for text, truth in samples:
        out = run(prompt_fn(text))
        label = normalize_label(out)
        preds.append(label); golds.append(truth)
        rows.append((text, truth, out, label))
    acc = sum(p==g for p,g in zip(preds,golds)) / len(golds)
    cm = Counter((golds[i], preds[i]) for i in range(len(golds)))
    return acc, cm, rows

def pretty_confusion(cm):
    cats = ["Positive","Negative","Neutral"]
    header = "          " + "  ".join(f"{c:>8}" for c in cats)
    lines = [header]
    for g in cats:
        line = [f"{g:>8}"]
        for p in cats:
            line.append(f"{cm.get((g,p),0):>8}")
        lines.append("  ".join(line))
    return "\n".join(lines)

# --- Run all versions ---
versions = {
    "A_zero_shot": prompt_zero_shot,
    "B_one_shot": prompt_one_shot,
    "C_few_shot_json": prompt_few_shot_json,
    "D_bad_prompt": prompt_bad,
}

results = {}
for name, fn in versions.items():
    acc, cm, rows = evaluate(name, fn, data)
    results[name] = {"acc":acc, "cm":cm, "rows":rows}

for name, res in results.items():
    print(f"\n=== {name} ===")
    print(f"Accuracy: {res['acc']:.2f}")
    print(pretty_confusion(res["cm"]))

# --- Show errors (if any) for each version ---
for name, res in results.items():
    errs = [(t,g,o,l) for (t,g,o,l) in res["rows"] if g != l]
    if not errs:
        continue
    print(f"\n--- Mistakes in {name} ---")
    for text, gold, raw, norm in errs:
        print(f"Text: {text}\nGold: {gold} | Raw: {raw!r} | Parsed: {norm}\n")


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Checking for Rust toolchain....
      Rust not found, installing into a temporary directory
      Python reports SOABI: cp313-win_amd64
      Computed rustc target triple: x86_64-pc-windows-msvc
      Installation directory: C:\Users\User\AppData\Local\puccinialin\puccinialin\Cache
      
      Installing rust to C:\Users\User\AppData\Local\puccinialin\puccinialin\Cache\rustup
      warn: installing msvc toolchain without its prerequisites
      info: profile set to 'minimal'
      info: default host triple is x86_64-pc-windows-msvc
      info: syncing channel updates for 'stable-x86_64-pc-windows-msvc'
      info: latest update on 2025-08-07, rust version 1.89.0 (29483883e 2025-08-04)
      info: downloading component 'cargo'
      info: downloading component 'rust-std'
      info: downloading component 'rustc'
      info: install

# One‑Page Report (Findings & Reflection)

Objective.
We compared four prompt designs for review‑sentiment classification using flan‑t5‑small:
(A) zero‑shot instruction, (B) one‑shot exemplar, (C) few‑shot JSON with three exemplars, and (D) an intentionally bad prompt with inconsistent instructions.

Method.
We evaluated on a small balanced set of 12 short reviews labeled Positive/Negative/Neutral. Metrics: overall accuracy + confusion matrices. We normalized model outputs to one of the three labels (and parsed JSON when provided).

Results (typical behavior).

A: Zero‑shot (instruction‑only). Usually decent, but occasional drift (e.g., extra words). Accuracy is moderate; most errors come from borderline/ambiguous statements (“fine for the price…”, “usable, partner likes it more”).

B: One‑shot. Accuracy improves. The exemplar strongly biases outputs toward exact, one‑word labels and reduces verbosity. Still, borderline examples can flip between Neutral and Positive/Negative.

C: Few‑shot JSON (3 shots). Most robust. The clear schema and exemplars reduce both hallucinations and format errors. Parsing is reliable, and decisions on “ambiguous” items stabilize. Typically the best confusion matrix (fewer false positives on Neutral).

D: Bad prompt. Performance degrades. Inconsistencies (“positive or negative” but later “sometimes neutral”), plus punctuation and “explanation welcome” encourages long-form text. This produces more parsing issues and label drift (more misclassifications and noisy outputs).

What didn’t work well.

Inconsistent label sets (saying only Positive/Negative but showing Neutral) and trailing punctuation (“Negative.”) increased drift.

Allowing explanations invited verbosity and made parsing harder.

Soft or hedged instructions (“maybe,” “best you think”) reduced format fidelity.

What worked well.

A crisp format contract (one word or strict JSON) stated before the examples.

User→assistant few‑shot pairs that exactly match the desired style.

A small, diverse set of exemplars: clearly positive, clearly negative, and borderline/ambiguous (Neutral) cases.

A normalizer to map outputs into canonical labels (and a JSON parser when applicable).

Lessons learned.

Shots teach format at inference time; even one‑shot significantly improves adherence.

Few‑shot + schema (JSON) is best when you need reliability for downstream code.

Be consistent: instruction label set, exemplars, and outputs must align.

Keep prompts minimal and deterministic: avoid extra prose, set “no extra text,” and specify the exact allowed labels.

Add a small client‑side guardrail (normalizer/JSON parser) to turn “messy but correct” outputs into clean labels.